In [1]:
import pandas as pd
import json
import numpy as np

# read data

In [3]:
book_df = pd.read_csv('ratings.csv')

In [4]:
print(book_df)

          userId  movieId  rating   timestamp
0              1       17     4.0   944249077
1              1       25     1.0   944250228
2              1       29     2.0   943230976
3              1       30     5.0   944249077
4              1       32     5.0   943228858
...          ...      ...     ...         ...
32000199  200948    79702     4.5  1294412589
32000200  200948    79796     1.0  1287216292
32000201  200948    80350     0.5  1294412671
32000202  200948    80463     3.5  1350423800
32000203  200948    87304     4.5  1350423523

[32000204 rows x 4 columns]


In [5]:
meta_df = pd.read_csv('movies.csv')


In [6]:
print(book_df.columns)
print(meta_df.columns)

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Index(['movieId', 'title', 'genres'], dtype='object')


# add meta info


In [8]:
links = pd.read_csv('links.csv')
basics = pd.read_csv('title.basics.tsv', sep = '\t')

/tmp/ipykernel_32850/2529126186.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv('/data/shith/datasets2024/cold/ml-32m/title.basics.tsv', sep = '\t')


In [10]:
print(links.head(5))

   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


In [18]:
def process_idstr(x):
    idstr = 'tt'
    if len(str(x)) < 7:
        for i in range(7 - len(str(x))):
            idstr += '0'
    idstr += str(x)
    return idstr

In [19]:
links['tconst'] = links['imdbId'].apply(process_idstr)

In [20]:
links_all = pd.merge(links, basics, how = 'left', on = 'tconst')

In [32]:
print(links_all.columns, meta_df.columns)

Index(['movieId', 'imdbId', 'tmdbId', 'tconst', 'titleType', 'primaryTitle',
       'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes',
       'genres'],
      dtype='object') Index(['movieId', 'title', 'genres'], dtype='object')


In [33]:
meta_df = pd.merge(meta_df, links_all, how =  'left', on = 'movieId')


In [35]:
meta_df['genres'] = meta_df['genres_x']

In [40]:
print(meta_df.head(100))

    movieId                                 title  \
0         1                      Toy Story (1995)   
1         2                        Jumanji (1995)   
2         3               Grumpier Old Men (1995)   
3         4              Waiting to Exhale (1995)   
4         5    Father of the Bride Part II (1995)   
..      ...                                   ...   
95       97               Hate (Haine, La) (1995)   
96       98                       Shopping (1994)   
97       99  Heidi Fleiss: Hollywood Madam (1995)   
98      100                      City Hall (1996)   
99      101                  Bottle Rocket (1996)   

                                       genres_x  imdbId   tmdbId     tconst  \
0   Adventure|Animation|Children|Comedy|Fantasy  114709    862.0  tt0114709   
1                    Adventure|Children|Fantasy  113497   8844.0  tt0113497   
2                                Comedy|Romance  113228  15602.0  tt0113228   
3                          Comedy|Drama|Romance

# save pkl 

In [38]:
print(book_df.columns)

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


In [41]:
book_small_df = book_df.copy()[['userId', 'movieId', 'rating', 'timestamp']]

In [42]:
book_small_df.columns = ['user_id',  'parent_asin', 'rating',  'timestamp']

In [43]:
book_small_df.drop_duplicates(inplace=True)

In [44]:
book_small_df.to_hdf('./small_Books.hdf',key = '1')
book_small_df.to_pickle('./small_Books.pkl')

/data/shith/anaconda3/envs/pytorch/lib/python3.10/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [59]:
print(book_small_df.shape[0])
print(book_small_df.head(5))

32000204
   user_id  parent_asin  rating  timestamp
0        1           17     4.0  944249077
1        1           25     1.0  944250228
2        1           29     2.0  943230976
3        1           30     5.0  944249077
4        1           32     5.0  943228858


In [46]:
print(meta_df.columns)

Index(['movieId', 'title', 'genres_x', 'imdbId', 'tmdbId', 'tconst',
       'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres_y', 'genres'],
      dtype='object')


In [48]:
meta_small_df = meta_df.copy()[[ 'title', 'movieId', 'genres', 'imdbId', 'tmdbId',
                      'startYear',  'titleType',   'runtimeMinutes', 'endYear',    ]]

In [50]:
print(meta_small_df.head(5))
print(meta_small_df.columns)

                                title  movieId  \
0                    Toy Story (1995)        1   
1                      Jumanji (1995)        2   
2             Grumpier Old Men (1995)        3   
3            Waiting to Exhale (1995)        4   
4  Father of the Bride Part II (1995)        5   

                                        genres  imdbId   tmdbId startYear  \
0  Adventure|Animation|Children|Comedy|Fantasy  114709    862.0      1995   
1                   Adventure|Children|Fantasy  113497   8844.0      1995   
2                               Comedy|Romance  113228  15602.0      1995   
3                         Comedy|Drama|Romance  114885  31357.0      1995   
4                                       Comedy  113041  11862.0      1995   

  titleType runtimeMinutes endYear  
0     movie             81      \N  
1     movie            104      \N  
2     movie            101      \N  
3     movie            124      \N  
4     movie            106      \N  
Index(['title'

In [53]:
meta_small_df.columns = ['title', 'parent_asin', 'genres', 'imdbId', 'tmdbId', 'startYear',
       'titleType', 'runtimeMinutes', 'endYear']

In [54]:
meta_small_df['publish_time'] = meta_small_df['startYear']

In [61]:
print(meta_small_df[meta_small_df['publish_time'] == '2023'])

                              title  parent_asin  \
54212                      Empire V       187647   
66506              Sound of Freedom       215173   
71721               Hopewell (2020)       231425   
72599       The Malibu Tapes (2019)       235259   
79100  The Everlasting Story (2021)       264900   
...                             ...          ...   
87574               Comeback (2023)       292617   
87576          Flaming Cloud (2023)       292625   
87577      All Fun and Games (2023)       292627   
87581    Shelter in Solitude (2023)       292737   
87584     Race to the Summit (2023)       292757   

                             genres    imdbId     tmdbId startYear titleType  \
54212                       Fantasy   5183894   513505.0      2023     movie   
66506            (no genres listed)   7599146   678512.0      2023     movie   
71721                      Thriller   2315938   762640.0      2023     movie   
72599                        Horror   2488666   420637.

In [55]:
meta_small_df.to_pickle('./small_meta_Books.pkl')

In [56]:
print(meta_small_df['title'].unique().shape)

(87382,)


In [57]:
print(meta_small_df['title'])

0                          Toy Story (1995)
1                            Jumanji (1995)
2                   Grumpier Old Men (1995)
3                  Waiting to Exhale (1995)
4        Father of the Bride Part II (1995)
                        ...                
87580             The Monroy Affaire (2022)
87581            Shelter in Solitude (2023)
87582                           Orca (2023)
87583                The Angry Breed (1968)
87584             Race to the Summit (2023)
Name: title, Length: 87585, dtype: object
